## Problem 23

Works correctly according to the sample data given by rosalind but I can't seem to pass the actual tests.

In [71]:
from collections import defaultdict
import numpy as np
import pandas as pd

    
def est_t_matrix(path, states):
    # initialize matrix and final matrix
    shape = (len(states), len(states))
    t_matrix = pd.DataFrame(np.full(shape, 0), index=states, columns=states)
    final_t_matrix = pd.DataFrame(np.full(shape, 0), index=states, columns=states)
    
    # add up transitions
    for x in range(len(path)-1):
        initial = path[x]
        next = path[x+1]
        t_matrix.loc[initial, next] += 1
        
    # normalize table into a final table
    for row in states:
        row_sum = t_matrix.loc[row].sum(axis=0)

        for column in states:
            if row_sum: # check if row_sum isn't 0
                final_t_matrix.loc[row, column] = t_matrix.loc[row, column] / row_sum
            else:
                final_t_matrix.loc[row, column] = 1 / len(states)
    
    return final_t_matrix


def est_emi_matrix(path, emi_path, states, emi_states):
    shape = (len(states), len(emi_states))
    emi_matrix = pd.DataFrame(np.full(shape, 0), index=states, columns=emi_states)
    final_emi_matrix = pd.DataFrame(np.full(shape, 0), index=states, columns=emi_states)
    
    for state, emitted in zip(path, emi_path):
        emi_matrix.loc[state, emitted] += 1
    
    for row in states:
        row_sum = emi_matrix.loc[row].sum(axis=0)
        
        for column in emi_states:
            if row_sum:
                final_emi_matrix.loc[row, column] = emi_matrix.loc[row, column] / row_sum
            else:
                final_emi_matrix.loc[row,column] = 1 / len(states)
    
    return final_emi_matrix

In [116]:
with open("rosalind_ba10h.txt") as f:
    lines = [line.strip() for line in f]

    
observed_path = lines[0]
emi_states = lines[2].split()
path = lines[4]
states = lines[6].split()

print(est_t_matrix(path, states))
print('--------')
print(est_emi_matrix(path, observed_path, states, emi_states))

          A         B      C
A  0.281250  0.343750  0.375
B  0.428571  0.371429  0.200
C  0.281250  0.343750  0.375
--------
          x         y         z
A  0.333333  0.242424  0.424242
B  0.314286  0.371429  0.314286
C  0.531250  0.281250  0.187500


## Problem 24
Pretty simple as it just reuses functions we've already created.

In [118]:
import numpy as np
import pandas as pd

def viterbi_path(emi_path, states, t_matrix, emi_matrix):
    #hold scores and states for backtracking
    vit_table = pd.DataFrame(    np.full((len(states),len(emi_path)), 0)   , index=states)
    back_track = pd.DataFrame(   np.full((len(states),len(emi_path)), 0)   , index=states)
    
    for state in states: #initialize start scores
        vit_table.at[state,0] = (1/len(states)) * emi_matrix.loc[state, emi_path[0]]
    
    for time in range(1, len(emi_path)): #nested loop for calculating scores of every node
        for state in states:
            # prev_scores = [score_a/b/c_i-1 * weight]
            prev_scores = [vit_table.loc[prev_state][time-1] * t_matrix.loc[prev_state, state] * emi_matrix.loc[state, emi_path[time]] for prev_state in states]
            # fill in node with max of preceding scores
            vit_table.at[state,time] = max(prev_scores)
            # keep track of the previous node-state that gave the highest score
            back_track.at[state,time] = max(states, key=lambda prev_state: vit_table.loc[prev_state,time-1] * t_matrix.loc[prev_state, state] * emi_matrix.loc[state, emi_path[time]])
    
    # Append to a list, backtracking from the start node, then reversing the list for the right order
    
    #Find the max value of the last column and return its corresponding state
    vit_path = [max(states, key=lambda state: vit_table.loc[state, len(emi_path)-1])]
    for time in range(len(emi_path)-1, 0, -1):
        # uses the last list element to find the previous node from backtrack
        vit_path.append(back_track.loc[vit_path[-1],time])

    vit_path.reverse()
    
    return vit_path

In [119]:
def viterbi_learning(emi_path, t_matrix, emi_matrix, states, emi_states, iterations=100):
    # Hold matrices for repeated iteration
    t_matrix_iter = t_matrix
    emi_matrix_iter = emi_matrix
    
    for x in range(iterations):
        # Calculate path
        path = viterbi_path(emi_path, states, t_matrix_iter, emi_matrix_iter)
        
        # Feed path into estimation algorithms
        t_matrix_iter = est_t_matrix(path, states)
        emi_matrix_iter = est_emi_matrix(path, emi_path, states, emi_states)
        # repeat
    
    return t_matrix_iter, emi_matrix_iter



In [120]:
with open("rosalind_ba10i2.txt") as f:
    lines = [line.strip() for line in f]

iterations = int(lines[0])    
observed_path = lines[2]
emi_states = lines[4].split()
states = lines[6].split()

t_matrix = []
emi_matrix = []
for x in range(9, 9+len(states)):
    t_matrix.append(lines[x].strip().split()[1:])

for x in range(9+len(states)+2, 9+len(states)+2+len(states)):
    emi_matrix.append(lines[x].strip().split()[1:])

t_matrix22 = pd.DataFrame(t_matrix, index=states, columns=states).astype(float)
emi_matrix22 = pd.DataFrame(emi_matrix, index=states, columns=emi_states).astype(float)

for x in viterbi_learning(observed_path, t_matrix22, emi_matrix22, states, emi_states, iterations):
    print(x)

          A         B         C
A  0.047619  0.952381  0.000000
B  0.684211  0.315789  0.000000
C  0.333333  0.333333  0.333333
          x         y         z
A  0.000000  0.642857  0.357143
B  0.586207  0.000000  0.413793
C  0.333333  0.333333  0.333333
